<a href="https://colab.research.google.com/github/Kimyongju40/Kim.github.io/blob/main/n123a_Hypothesis_Test_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/89746244-a6e2a200-daf3-11ea-9314-7863d544d73c.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 3*

---

# 📝 Assignment

# **🌱 기본 과제**

오늘 lecture note에서는 두 집단의 평균을 비교하는 가설 검정 과정을 부트스트래핑을 이용해 표본 평균 분포를 구현하여 통계적 유의미함을 살펴보았습니다. 

**T-test** 메소드에는 이러한 과정이 built-in 되어있기 때문에 우리는 동일한 계산을 **T-test** 메소드를 통해 간단히 구현해 볼 수 있습니다. 

## **Part 1. One-Sample T-test** 
한 샘플의 평균값과 특정값(모수)을 비교하는 검정 방법입니다. <br>
이를 양측검정을 통해 시행하고자 한다면 귀무가설과 대립가설은 다음과 같이 세울 수 있습니다. 

$H_0$ : $\mu_{sample} = \mu_{population}$

$H_a$ : $\mu_{sample} ≠ \mu_{population}$ 

### **1-1. 데이터셋 준비**
- 다음 데이터셋을 불러옵니다. <br>
  https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/seoul_tree/seoul_tree.txt
  - 데이터는 `tap`으로 구분되어 있습니다. 
  - `skiprows=1`로 설정하여 첫 번째 row가 헤더가 되도록 지정합니다. 
- `자치구`가 `서울시`, `공원녹지사업소`, `시설관리공단`인 데이터는 제외합니다. 
- `이팝나무` column만 추출하여 `df_sub`에 저장하세요. 
  - `df_sub`은 Series 형태입니다. 
- `df_sub`의 index를 재설정 하세요. 

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
df= pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/seoul_tree/seoul_tree.txt',sep='\t', skiprows=1)
df.head()

,기간,자치구,노선수,수량,은행나무,양버즘나무,느티나무,왕벚나무,은단풍나무,메타세콰이어,...,튜울립나무,칠엽수,포플러,느릅나무,살구나무,무궁화,소나무,목련,이팝나무,기타
0,2019,서울시,"1,412","307,351","108,050","62,474","37,473","34,150",545,"5,171",...,948,"1,533",7,504,234,"2,374","4,539",378,"20,088","20,112"
1,2019,종로구,70,"7,329","4,072","1,071",619,421,-,1,...,-,17,-,-,-,-,239,-,711,29
2,2019,중구,48,"8,250","3,640",162,460,289,-,-,...,7,-,-,-,21,46,"2,162",-,470,817
3,2019,용산구,39,"7,835","4,444","1,422",373,262,-,-,...,13,16,-,-,39,-,51,126,198,780
4,2019,성동구,38,"8,120","1,090","2,401","1,975",602,29,48,...,-,1,3,59,-,-,53,-,709,646


In [38]:
df_sub = df.query("자치구 not in ['서울시', '공원녹지사업소', '시설관리공단']")["이팝나무"].reset_index(drop=True)
df_sub

0       711
1       470
2       198
3       709
4       760
5     1,176
6       449
7       853
8       575
9        47
10      142
11      443
12      493
13    1,280
14      784
15    1,426
16    1,082
17      184
18    1,281
19      397
20      249
21      446
22      768
23    1,651
24      642
Name: 이팝나무, dtype: object

##### **Q1. `df_sub`의 row의 총 개수를 쓰세요.**

In [41]:
df_sub.shape[0]

25

### **1-2. 데이터 타입 변환**
- `이팝나무` column의 데이터 타입을 확인한 후, 적절한 데이터 타입으로 변환하세요.
  - 적절한 데이터 타입으로 변환하기 위해 `.replace()` 함수를 활용해보세요.

##### **Q2. `이팝나무` column의 데이터 타입으로 적절한 것은 무엇인가요?**

In [47]:
df_sub = df_sub.replace(",","").astype(int)

In [48]:
df_sub.dtype

dtype('int64')

### **1-3. 표본 생성 및 가설 검정**
- [`np.random.choice()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)를 사용해 10개의 구의 `이팝나무` 데이터를 중복을 허용하여 추출합니다. 
    - `np.random.seed(123)`으로 설정하세요.
- 서울시 구별 `이팝나무` 데이터를 모집단으로 가정하였을 때, 10개의 구를 추출한 `이팝나무` 수의 평균과 모집단의 평균을 비교하는 가설검정을 One-Sample T-test를 통해 시행하세요.      
  - scipy 라이브러리의 [`.ttest_1samp()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html)을 사용합니다.
- `.pvalue`를 사용하여 가설검정 결과의 P-value를 `pv1`에 저장하세요.

##### **Q3. 모집단(서울시 구별 이팝나무 데이터)의 평균은 얼마인지 소수점 둘째 자리까지 구하세요.**

In [49]:
round(df_sub.mean(),2)

688.64

##### **Q4. 표본(10개의 구를 추출한 이팝나무 데이터)의 평균은 얼마인지 소수점 첫째 자리까지 구하세요.**

In [62]:
import random
from scipy import stats

np.random.seed(123)

이팝나무_10 = np.random.choice(df_sub, 10, replace=True, p=None)

0.09699658599768855

In [63]:
round(이팝나무_10.mean(),1)

479.7

##### **Q5. 가설 검정 결과 P-value를 해석한 것으로 알맞은 것은 무엇인가요?**
- 서울시 구별 이팝나무 수의 평균과 10개의 구를 샘플링한 이팝나무 수의 평균은 다르다고 볼 수 없다. 
- 서울시 구별 이팝나무 수의 평균과 10개의 구를 샘플링한 이팝나무 수의 평균은 같다고 볼 수 없다. 
- 서울시 구별 이팝나무 수의 평균은 10개의 구를 샘플링한 이팝나무 수의 평균보다 크다. 
- 서울시 구별 이팝나무 수의 평균은 10개의 구를 샘플링한 이팝나무 수의 평균보다 작다. 

In [64]:
이팝나무_mean =  df_sub.mean()
resurt = stats.ttest_1samp(이팝나무_10, 이팝나무_mean)
pv1 = resurt.pvalue
pv1

0.09699658599768855

# **🥇 도전 과제**

## **Part 2. Two-Sample T-test**


두 샘플의 평균값을 비교하는 검정 방법입니다. <br>
이를 양측검정을 통해 시행하고자 한다면 귀무가설과 대립가설은 다음과 같이 세울 수 있습니다. 

$H_0$ : $\mu_{sample1} = \mu_{sample2}$

$H_a$ : $\mu_{sample1} ≠ \mu_{sample2}$

In [ ]:
# 아래의 코드를 실행해 주세요. 업그레이드 완료 후에는 런타임을 재시작해야 합니다. 
!pip install scipy --upgrade

In [ ]:
# 버전 확인 
import scipy

scipy.__version__

'1.7.3'

### **2-1. 데이터셋 준비**
- **`서울시`, `공원녹지사업소`, `시설관리공단` 데이터는 제외하세요.** 
- **`느티나무`, `왕벚나무` column만 추출하여 각각 `zelkova`과 `cherry`에 저장하세요.** 

In [ ]:
# 이곳에 코드를 작성하세요.

### **2-2. 데이터 타입 변환**
**`느티나무`, `왕벚나무` column의 데이터 타입을 `int`로 변환하세요.**

In [ ]:
# 이곳에 코드를 작성하세요.

### **2-3. 가설 검정**
단측 검정(One-tailed test)을 사용하여 평균 `느티나무` 수와 평균 `왕벚나무` 수를 비교하는 가설검정을 시행하세요. (단, 서울시 구별 `느티나무` 및 `왕벚나무` 데이터는 표본입니다.) 
- **scipy 라이브러리의 [`.ttest_ind()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)를 사용합니다.**
  - One-tailed test를 진행하기 위해서는 `alternative` 파라미터를 설정해야 합니다. 
- **'평균 느티나무 수가 왕벚나무 수보다 통계적으로 유의미하게 크다'는 가설검정 결과의 P-value를 `pv2`에 저장하세요.**
   - 이에 알맞은 귀무가설과 대립가설을 설정합니다. 
- **'평균 왕벚나무 수가 느티나무 수보다 통계적으로 유의미하게 크다'는 가설검정 결과의 P-value를 `pv3`에 저장하세요.** 
  - 이에 알맞은 귀무가설과 대립가설을 설정합니다. 
- **평균 `느티나무` 수와 평균 `왕벚나무` 수를 비교하는 가설검정 결과를 `ans`에 저장하세요. 이때 `ans`는 다음 세 가지 중 하나입니다.** 
  - 느티나무 (평균 느티나무 수가 왕벚나무 수보다 통계적으로 유의하게 큰 경우)
  - 왕벚나무 (평균 왕벚나무 수가 느티나무 수보다 통계적으로 유의하게 큰 경우)
  - 같다. (통계적으로 유의미한 차이가 없다.) 

In [ ]:
# 이곳에 코드를 작성하세요.